#### Challenge 46: RSA parity oracle

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [22]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import base64
import cryptopals as cp

<div class="alert alert-block alert-info">   

<div class="alert alert-block alert-warning">
    
#### **When does this ever happen?**

This is a bit of a toy problem, but it's very helpful for understanding what RSA is doing (and also for why pure number-theoretic encryption is terrifying). Trust us, you want to do this before trying the next challenge. Also, it's fun.

</div>
</div>    

<div class="alert alert-block alert-info">   
    
Generate a 1024 bit RSA key pair.
    
</div>

In [23]:
e,d,n = cp.genRSA_keypair(1024)

<div class="alert alert-block alert-info">   

Write an oracle function that uses the private key to answer the question "is the plaintext of this message even or odd" (is the last bit of the message 0 or 1). Imagine for instance a server that accepted RSA-encrypted messages and checked the parity of their decryption to validate them, and spat out an error if they were of the wrong parity.

Anyways: function returning true or false based on whether the decrypted plaintext was even or odd, and nothing else.

</div>

In [24]:
def pt_is_odd(ciphertext):
    """Return True of plaintext is odd.  False if plaintext is even."""
    plaintext = pow(ciphertext, d, n)
    return((plaintext % 2) == 1)

<div class="alert alert-block alert-info">   
    
Take the following string and un-Base64 it in your code (without looking at it!) and encrypt it to the public key, creating a ciphertext:

`VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==`

</div>

In [25]:
s = 'VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ=='
s_int = int(base64.b64decode(s).hex(), 16)
ciphertext = pow(s_int, e, n)

<div class="alert alert-block alert-info">   

With your oracle function, you can trivially decrypt the message.

Here's why:

- RSA ciphertexts are just numbers. You can do trivial math on them. You can for instance multiply a ciphertext by the RSA-encryption of another number; the corresponding plaintext will be the product of those two numbers.
- If you double a ciphertext (multiply it by `(2**e)%n)`, the resulting plaintext will (obviously) be either even or odd.
- If the plaintext after doubling is even, doubling the plaintext didn't wrap the modulus --- the modulus is a prime number. That means the plaintext is less than half the modulus.

You can repeatedly apply this heuristic, once per bit of the message, checking your oracle function each time.

Your decryption function starts with bounds for the plaintext of `[0,n]`.

Each iteration of the decryption cuts the bounds in half; either the upper bound is reduced by half, or the lower bound is.

After `log2(n)` iterations, you have the decryption of the message.

Print the upper bound of the message as a string at each iteration; you'll see the message decrypt "hollywood style".

Decrypt the string (after encrypting it to a hidden private key) above.

</div>

In [26]:
def print_it(x):
    
    hex_x = hex(x)[2:]
    if len(hex_x) % 2:
        hex_x = '0' + hex_x
    print(bytes.fromhex(hex_x))

In [26]:
DISPLAY_PROGRESS = False

lower_bound = 0
upper_bound = n

tmp = ciphertext

while (upper_bound - lower_bound) > 0:
    tmp = (tmp * pow(2, e, n)) % n
    if pt_is_odd(tmp):
        #lower_bound += (upper_bound - lower_bound) // 2
        lower_bound = (lower_bound + upper_bound) // 2
    else:        
        #upper_bound -= (upper_bound - lower_bound) // 2
        upper_bound = (lower_bound + upper_bound) // 2
        
    if DISPLAY_PROGRESS:
        
        print_it(upper_bound)
       
print_it(upper_bound)

b"That's why I found you don't play around with the Funky Cold Medin "


---
TODO:

Something's not quite right -- not properly identifying the last character... Not sure why... Rounding issue?   

---

Let's try to understand what's going on here a little better.  Let's choose some small RSA parameters:

In [69]:
p = 11
q = 17
n = p*q
et = (p-1)*(q-1)
e=3
d = cp.invmod(e, et)
print(e, d, n)

Make sure my Simple RSA works properly:

In [70]:
x = 7
assert(((((x**e)%n)**d) % n) == x)

Now, demonstrate that math on the ciphertext ==> same math on the plaintexts:

In [71]:
pt = 3
ct = (pt**e) % n

If I multiply the an "encrypted" 2, it will also multiply the plaintext by 2:

In [72]:
ct = (ct * (2**e % n))
pt = (ct**d) % n
print(pt)

6


The challenge says that:
    
    If the plaintext after doubling is even, doubling the plaintext didn't wrap the modulus --- the modulus is a prime number. That means the plaintext is less than half the modulus.
    
Let's try this on our little plaintext.

In [86]:
pt = 3
ct = (pt**e) % n
lower_bound = 0
upper_bound = n

while (upper_bound != lower_bound):
    
    print(f"Lower bound = {lower_bound}")
    print(f"Upper Bound = {upper_bound}")
    ct = (ct * (2**e % n) % n)
    pt_odd = pt_is_odd(ct)
    print(f"Is PT odd? {pt_odd}")
    # Peak at the current pt:
    print(f"Current 'PT' = {ct**d % n}")
    if pt_odd:
        lower_bound += (upper_bound - lower_bound) // 2
    else:        
        upper_bound -= (upper_bound - lower_bound) // 2
        
print(f"Lower bound = {lower_bound}")
print(f"Upper Bound = {upper_bound}")
print(f"Current 'PT' = {ct**d % n}")

Lower bound = 0
Upper Bound = 187
Is PT odd? False
Current 'PT' = 6
Lower bound = 0
Upper Bound = 94
Is PT odd? False
Current 'PT' = 12
Lower bound = 0
Upper Bound = 47
Is PT odd? False
Current 'PT' = 24
Lower bound = 0
Upper Bound = 24
Is PT odd? False
Current 'PT' = 48
Lower bound = 0
Upper Bound = 12
Is PT odd? False
Current 'PT' = 96
Lower bound = 0
Upper Bound = 6
Is PT odd? True
Current 'PT' = 5
Lower bound = 3
Upper Bound = 6
Is PT odd? False
Current 'PT' = 10
Lower bound = 3
Upper Bound = 5
Is PT odd? False
Current 'PT' = 20
Lower bound = 3
Upper Bound = 4
Is PT odd? False
Current 'PT' = 40
Lower bound = 3
Upper Bound = 4
Is PT odd? False
Current 'PT' = 80
Lower bound = 3
Upper Bound = 4
Is PT odd? False
Current 'PT' = 160
Lower bound = 3
Upper Bound = 4
Is PT odd? True
Current 'PT' = 133
Lower bound = 3
Upper Bound = 4
Is PT odd? True
Current 'PT' = 79
Lower bound = 3
Upper Bound = 4
Is PT odd? False
Current 'PT' = 158
Lower bound = 3
Upper Bound = 4
Is PT odd? True
Current 'P

KeyboardInterrupt: 

Addition is different...addition in GF(2) is XOR:

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)